In [3]:
import psycopg2
import pandas as pd
import matplotlib.pyplot as plt
import io
import numpy as np

In [4]:
data = pd.read_csv("result/cam_115_preset_1.csv")
data.head()

,timestamp,avg_speed,cam.preset,avg_speed_bus
0,2021-12-10 00:20:00,15.0,1.0,NaN
1,2021-12-10 00:25:00,24.0,1.0,NaN
2,2021-12-10 00:30:00,NaN,NaN,NaN
3,2021-12-10 00:35:00,NaN,NaN,NaN
4,2021-12-10 00:40:00,NaN,NaN,NaN


In [5]:
database = ''
user = 'postgres'
password = '00'
host = 'localhost'
port = '5432'
conn_string = "host='%s' dbname='%s' user='%s' password='%s' port='%s'" %(host, database, user, password, port)
conn = psycopg2.connect(conn_string)
conn.close()

In [6]:
conn = psycopg2.connect(conn_string)

cur = conn.cursor()

# Drop the table if it already exists
cur.execute("DROP table IF EXISTS test_data;")

# load data into Pandas dataframe
df2 = pd.read_csv("result/cam_115_preset_1.csv")
df = df2.head(200)

# only select the time column and `ts_7' column
df_db = df[['timestamp', 'avg_speed']]

# create table
create_table = "CREATE  TABLE test_data(\
   timestamp timestamp PRIMARY KEY,\
   avg_speed double precision);"
cur.execute(create_table)

# Populate table using pandas
output = io.StringIO()
df_db.to_csv(output, sep='\t', header=False, index=False)
output.seek(0)
cur.copy_from(output, 'test_data', null="")

# close connection
conn.commit()
conn.close()
print('Table created and Loaded')



Table created and Loaded


In [7]:
conn = psycopg2.connect(conn_string)
df = pd.read_sql_query("select * from test_data limit 10;", conn)
conn.close()
df

,timestamp,avg_speed
0,2021-12-10 00:20:00,15.0
1,2021-12-10 00:25:00,24.0
2,2021-12-10 00:30:00,NaN
3,2021-12-10 00:35:00,NaN
4,2021-12-10 00:40:00,NaN
5,2021-12-10 00:45:00,46.0
6,2021-12-10 00:50:00,9.0
7,2021-12-10 00:55:00,NaN
8,2021-12-10 01:00:00,NaN
9,2021-12-10 01:05:00,NaN


In [8]:
conn.close()
conn = psycopg2.connect(conn_string)
cursor = conn.cursor()
cursor.execute("""select create_pindex('test_data', 'timestamp','{"avg_speed"}','pindex', k =>4);""")
conn.commit()
conn.close()

In [9]:
conn = psycopg2.connect(conn_string)
df = pd.read_sql_query("select * from list_pindices();", conn)
df

,index_name,value_columns,relation,time_column,initial_timestamp,last_timestamp,agg_interval,uncertainty_quantification
0,pindex,[avg_speed],test_data,timestamp,2021-12-10 00:20:00,2021-12-10 16:55:00,00:05:00,True
1,pindex2,[velocity],newtable,time_stamp,2020-05-04 00:00:00,2020-05-05 13:44:00,00:01:00,True
2,pindex_timenew2,[velocity],pgfutter2,time,1,41943,1 units,True
3,pindex_timenew4,[avg_speed],pgfutter3,time,1,200,1 units,True
4,mixturets_var_pindex,[ts],mixturets_var,time,0,99999,1 units,True
5,basic_pindex,[ts],ts_basic_ts_5_5,time,2012-10-01 00:00:00,2012-10-06 18:53:15,00:00:05,True
6,pindex1,[ts_7],mixturets2,time,0,100000,1 units,True
7,pindex_timenew,[velocity],pgfutter2,time,1,41943,1 units,True
8,mixturets2_pindex,[ts],mixturets2,time,0,100000,1 units,True


In [10]:
df = pd.read_sql_query("select * from pindices_stat() where index_name = 'pindex';", conn)
df

,index_name,column_name,number_of_observations,number_of_trained_models,imputation_score,forecast_score,test_forecast_score
0,pindex,avg_speed,200,1,0.866239,0.968836,None


In [11]:
predict_start_time = pd.Timestamp('2021-12-10 04:00:00')
predict_start_time_str = predict_start_time.strftime('%Y-%m-%d %H:%M:%S')

predict_finish_time = pd.Timestamp('2021-12-10 08:00:00')
predict_finish_time_str = predict_finish_time.strftime('%Y-%m-%d %H:%M:%S')

df = pd.read_sql_query('''select * from predict('test_data','avg_speed','2021-12-11 04:00:00', '2021-12-11 08:00:00','pindex');''', conn)
df

,prediction,lb,ub
0,-8.767550e+34,-8.767550e+34,-8.767550e+34
1,1.576500e+35,1.576500e+35,1.576500e+35
2,-2.834717e+35,-2.834717e+35,-2.834717e+35
3,5.097128e+35,5.097128e+35,5.097128e+35
4,-9.165186e+35,-9.165186e+35,-9.165186e+35
5,1.647999e+36,1.647999e+36,1.647999e+36
6,-2.963281e+36,-2.963281e+36,-2.963281e+36
7,5.328299e+36,5.328299e+36,5.328299e+36
8,-9.580856e+36,-9.580856e+36,-9.580856e+36
9,1.722741e+37,1.722741e+37,1.722741e+37


In [15]:
conn = psycopg2.connect(conn_string)
cur = conn.cursor()
cur.execute("ALTER TABLE test_data DROP COLUMN IF EXISTS timestamp;")
conn.commit()

In [16]:
cur = conn.cursor()
cur.execute(" ALTER TABLE test_data ADD COLUMN time SERIAL PRIMARY KEY;")
conn.commit()

In [28]:
cur = conn.cursor()
df = pd.read_sql_query('''SELECT * FROM test_data order by time desc limit 10;''', conn)
df


,avg_speed,time
0,10.50,200
1,15.00,199
2,38.00,198
3,11.00,197
4,19.00,196
5,0.00,195
6,21.00,194
7,18.75,193
8,25.00,192
9,9.00,191


In [24]:
cur = conn.cursor()
cur.execute("""select create_pindex('test_data', 'time','{"avg_speed"}','pindex2', k =>4);""")
conn.commit()

In [31]:
df = pd.read_sql_query('''select * from predict('test_data','avg_speed', 202, 'pindex2');''', conn)
df

DatabaseError: Execution failed on sql 'select * from predict('test_data','avg_speed', 202, 'pindex2');': ValueError: could not broadcast input array from shape (2,) into shape (3,)
CONTEXT:  Traceback (most recent call last):
  PL/Python function "predict", line 13, in <module>
    prediction,interval = get_prediction( index_name_, table_name, value_column,  plpyimp(plpy), t, uq, projected = projected, uq_method = uq_method, c = c)
  PL/Python function "predict", line 225, in get_prediction
  PL/Python function "predict", line 493, in _get_forecast_range
PL/Python function "predict"
